# Requirements

In [7]:
import imaplib
import email
from email import policy
import os
import pandas as pd
from email.header import decode_header
import re
from bs4 import BeautifulSoup
import csv

In [8]:
import sys
sys.path.append(os.path.abspath('../../src'))
from helper_functions.path_resolver import DynamicPathResolver

dpr = DynamicPathResolver(marker="README.md")
paths = dpr.structure

Project Root: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_detection


In [10]:
raw_dir = dpr.get_folder_path_from_namespace(paths.data.raw)

# Mails from S., M., A.

### Mail extraction

##### IMAP connect

In [ ]:
IMAP_SERVER = "imap.web.de"
EMAIL_ACCOUNT = ''
EMAIL_PASSWORD = ''

In [5]:
mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)

('OK', [b'LOGIN completed'])

##### Functions

In [6]:
def extract_email_address(sender):
    email_match = re.search(r'<(.+?)>', sender)
    return email_match.group(1) if email_match else sender 

In [7]:
def decode_mime_header(header_value):
    if header_value:
        decoded_parts = decode_header(header_value)
        decoded_text = []

        for part, encoding in decoded_parts:
            try:
                if isinstance(part, bytes):
                    decoded_text.append(part.decode(encoding or "utf-8", errors="replace"))  
                else:
                    decoded_text.append(part)
            except LookupError:  
                decoded_text.append(part.decode("utf-8", errors="replace"))  

        return " ".join(decoded_text)
    return "Unknown"

In [8]:
def clean_html_body(html_body):
    soup = BeautifulSoup(html_body, "html.parser")
    text = soup.get_text(separator="\n", strip=True)
    return text

In [ ]:
def fetch_emails(folder, label):
    mail.select(folder)
    status, messages = mail.search(None, "ALL")
    email_ids = messages[0].split()
    email_data = []

    for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, "(RFC822)")
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])

                print(f"Raw Subject: {msg['Subject']}")
                print(f"Decoded Subject: {decode_mime_header(msg['Subject'])}")

                # Decode headers
                subject = decode_mime_header(msg["Subject"]) or "No Subject"
                sender = decode_mime_header(msg["From"])
                sender = extract_email_address(sender)
                date = msg["Date"]
                body = ""

                # Only plain text
                if msg.is_multipart():
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        content_disposition = part.get("Content-Disposition", "")

                        # Skip attachmenets
                        if content_type == "text/plain" and "attachment" not in content_disposition:
                            body = part.get_payload(decode=True).decode(errors="ignore")
                            break  
                else:
                    body = msg.get_payload(decode=True).decode(errors="ignore")

                email_data.append([label, date, sender, subject, body])

    return email_data


In [10]:
def save_to_csv(emails_data, path):
    file_exists = os.path.isfile(path)
    df = pd.DataFrame(emails_data, columns=["Label", "Date", "Sender", "Subject", "Body"])

    df.to_csv(
        path,
        mode='a',  
        header=not file_exists, 
        index=False,
        encoding="utf-8",  
        quoting=csv.QUOTE_MINIMAL, 
        quotechar='"', 
    )

    return df

##### Save extracted

In [ ]:
raw_data_path = os.path.join(raw_dir, 'mails_raw.csv')

In [12]:
get_all = False

In [ ]:
if get_all:
    # Unlabeled
    unlabeled_emails = fetch_emails("INBOX", -1)
    df_unlabeled = save_to_csv(unlabeled_emails, raw_data_path)
    print(f"Saved {len(df_unlabeled)} unlabeled emails to mails_raw.csv")

    # Legit
    legit_emails = fetch_emails("INBOX", 0)
    df_legit = save_to_csv(legit_emails, raw_data_path)
    print(f"Saved {len(df_legit)} legit emails to mails_raw.csv")

    # Spam
    spam_emails = fetch_emails("Spam", 1) 
    df_spam = save_to_csv(spam_emails, raw_data_path)
    print(f"Saved {len(df_spam)} spam emails to mails_raw.csv")

    mail.logout()

##### Load raw

In [15]:
raw_df = pd.read_csv(raw_data_path)
print(f"Total emails: {len(raw_df)}")
print(raw_df["Label"].value_counts()) 

Total emails: 6382
Label
-1    6310
 1      51
 0      21
Name: count, dtype: int64


### Clean mails

##### Remove html

In [9]:
def clean_and_save(df, save_path, filter):
    def clean_html(text):
        return BeautifulSoup(str(text), "html.parser").get_text(separator="\n", strip=True)
    
    if filter:
        df["Body"] = df["Body"].apply(clean_html)
        df.to_csv(save_path, index=False, encoding="utf-8")
        print(f"Cleaned dataset saved as {save_path}")


In [ ]:
cleaned_data_path = os.path.join(raw_dir, 'mails_cleaned.csv')

In [ ]:
clean_and_save(raw_df, cleaned_data_path, filter=True)

C:\Users\ilian\AppData\Local\Temp\ipykernel_6756\2188038012.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(str(text), "html.parser").get_text(separator="\n", strip=True)
C:\Users\ilian\AppData\Local\Temp\ipykernel_6756\2188038012.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  return BeautifulSoup(str(text), "html.parser").get_text(separator="\n", strip=True)


Cleaned dataset saved as c:\Users\ilian\Documents\Projects\git_projects\university\phishing_bert\data\raw\cleaned_mails.csv


##### Load cleaned

In [ ]:
cleaned_df = pd.read_csv(cleaned_data_path)
print(f"Total emails: {len(cleaned_df)}")
print(cleaned_df["Label"].value_counts()) 

Total emails: 6382
Label
-1    6310
 1      51
 0      21
Name: count, dtype: int64


### Label

##### Load to label

In [ ]:
labeled_data_path = os.path.join(raw_dir, 'mails_labeled.csv')

In [19]:
to_label_df = pd.read_csv(labeled_data_path)
print(f"Total emails: {len(to_label_df)}")
print(to_label_df["Label"].value_counts()) 

Total emails: 6382
Label
-1    6310
 1      51
 0      21
Name: count, dtype: int64


##### Auto label

In [22]:
def auto_label_emails(df, legit_senders, spam_senders, save_path):
    # Label legit emails (0)
    df.loc[df["Sender"].isin(legit_senders), "Label"] = 0
    legit_count = df["Label"].value_counts().get(0, 0)
    print(f"✅ Auto-labeled {legit_count} emails as LEGIT.")

    # Label spam emails (1)
    df.loc[df["Sender"].isin(spam_senders), "Label"] = 1
    spam_count = df["Label"].value_counts().get(1, 0)
    print(f"🚨 Auto-labeled {spam_count} emails as SPAM.")

    # Save
    df.to_csv(save_path, index=False, encoding="utf-8")
    print(f"🎉 All emails labeled and saved at: {save_path}")

    return df

In [23]:
known_legit_senders = [
    "service@paypal.com",
    "noreply@amazon.de",
    "support@web.de",
    "noreply@mail.kleinanzeigen.de",
    "neu@mailings.web.de",
    "mail@newsletterdirekt24.eu",
    "info@newsletter.agrar-fachversand.com",
    "mail@newsletterdirekt24.eu",
    "newsletter@yatego.com",
    "noreply@ebay-kleinanzeigen.de",
    "info@bestprovita.com",
    "ebay@reply.ebay.de",
    "ebay@ebay.com",
    "info@customer.autobutler.de",
    "info@agrar-fachversand.com",
]

In [24]:
known_spam_senders = [
    "info@nl.you-buy.net",
    "info@waytohearts.live",
    "admin@treffegirls.com",
    "noreply@mylove.ru",
    "noreply@liebesfun.de",
    "info@beboo.ru",
    "best-pair@fotostrana.ru",
    "user-events@fotostrana.ru",
    "gifts@fotostrana.ru",
    "noreply@fotostrana.ru",
    "status@fotostrana.ru",
    "noreply@znakomstva.ru",
    "events@lafh.org",
    "skg@lafh.org",
    "info@fastheart-connect.eu",
    "dave@cadamedia.ie",
    "k_nadin@i.ua",
    "redaktion@newsletter.lonelyplanet.de",
    "Mailer-Daemon@diplom54.ru",
    "info@magic-mob.com",
    "info@sendmenews.live",
    "info@flashingpanties.eu",
    "z@proton-m03.sarbc.ru",
    "service@ga.acmaildist.com",
    "Thomas.ehrlich5481616@rifsgo.xyz",
    "Stefan-Schubert6786126@meggod.xyz",
    "Stefan-Schubert1081211@sortgo.xyz",
    "Dieter.Kirchner4913521@ankito.xyz",
    "noreply@znakomstva.ru",
    "admin@gibmirsex.com",
    "info@hsr-europe.eu",
    "info@sluttyhour.live",
    "noreply@fpz.de",
    "info@promaster-msg.live",
    "dominik-schroeder@ziegel.de",
    "marina@kouch-stilist.ru",
    "noreply@swing-zone.com",
    "meeting@fotostrana.ru",
    ]

In [ ]:
to_label_df = auto_label_emails(to_label_df, known_legit_senders, known_spam_senders, labeled_data_path)
print(to_label_df["Label"].value_counts()) 

✅ Auto-labeled 1395 emails as LEGIT.
🚨 Auto-labeled 774 emails as SPAM.
🎉 All emails labeled and saved at: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_bert\data\raw\labeled_mails.csv
Label
-1    4213
 0    1395
 1     774
Name: count, dtype: int64


##### Manual label

In [ ]:
for index, row in to_label_df[to_label_df["Label"] == -1].iterrows():
    print("\n" + "=" * 50)
    print(f"📩 Sender: {row['Sender']}")
    print(f"📜 Subject: {row['Subject']}")

    label = input("Label this email (0 = legit, 1 = spam, -1 = skip): ")

    try:
        label = int(label)
        if label in [0, 1]:
            sender = row['Sender']
            to_label_df.loc[to_label_df['Sender'] == sender, 'Label'] = label
            print(f"✅ All emails from {sender} have been labeled as {'LEGIT' if label == 0 else 'SPAM'}.")

    except ValueError:
        print("Skipping email...")

    if index % 10 == 0:
        to_label_df.to_csv(labeled_data_path, index=False, encoding="utf-8")
        print("✅ Progress saved.")



📩 Sender: info@b7mailing.com
📜 Subject: Anmeldung bestätigen und bis zu 15€ Bonus geschenkt bekommen!
✅ All emails from info@b7mailing.com have been labeled as SPAM.

📩 Sender: info@harlotlocatrs.live
📜 Subject: Gute Nachricht kruk3 ! Jemand mag dich!
✅ All emails from info@harlotlocatrs.live have been labeled as SPAM.

📩 Sender: ebay@reply2.ebay.de
📜 Subject: Diesen 5 €-Gutschein bekommt nicht jeder, Sergej Kohl
✅ All emails from ebay@reply2.ebay.de have been labeled as LEGIT.

📩 Sender: mjaramillo@lafh.org
📜 Subject: 3 ways to jump right in
✅ All emails from mjaramillo@lafh.org have been labeled as SPAM.

📩 Sender: kruk3@web.de
📜 Subject: ООО «УК «Домсервис» (2) messages from Isla and Leo
✅ All emails from kruk3@web.de have been labeled as SPAM.

📩 Sender: webmaster@alghuwaif.com
📜 Subject: Al Ghuwaif Feedback
✅ All emails from webmaster@alghuwaif.com have been labeled as SPAM.

📩 Sender: webmaster@alghuwaif.com
📜 Subject: Al Ghuwaif Feedback
✅ All emails from webmaster@alghuwaif.co

# Mails from J. Mail Server

In [11]:
from email.parser import BytesParser

In [13]:
def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text(separator='\n', strip=True)


def extract_email_data(email_file):
    encodings = ['utf-8', 'windows-1252', 'iso-8859-1']
    
    for enc in encodings:
        try:
            with open(email_file, 'rb') as f:
                msg = BytesParser(policy=policy.default).parse(f)

            sender = msg.get("From", None)
            subject = msg.get("Subject", None)
            to = msg.get("To", None)
            date = msg.get("Date", None)

            email_text = ""
            if msg.is_multipart():
                for part in msg.iter_parts():
                    if part.get_content_type() == 'text/plain':
                        email_text = part.get_payload(decode=True).decode(enc, errors='replace')
                        break
            else:
                if msg.get_content_type() == 'text/plain':
                    email_text = msg.get_payload(decode=True).decode(enc, errors='replace')

            email_text = email_text.replace('\r\n', '\n').replace('\r', '\n')
            return sender, subject, to, date, email_text.strip() if email_text.strip() else None

        except Exception as e:
            print(f"Failed to read {email_file} with encoding {enc}: {e}")
    
    return None, None, None, None, None

In [14]:
def process_spam_folder(spam_folder_path, output_csv_path):
    email_data = []

    if not os.path.exists(spam_folder_path):
        print(f"Folder not found: {spam_folder_path}")
        return

    for root, dirs, files in os.walk(spam_folder_path):
        for file in files:
            email_file_path = os.path.join(root, file)
            sender, subject, to, date, body = extract_email_data(email_file_path)

            email_data.append([sender if sender else "", 
                               subject if subject else "", 
                               to if to else "", 
                               date if date else "", 
                               body if body else ""])

    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Sender', 'Subject', 'Receiver', 'Date', 'Body'])
        writer.writerows(email_data)

    print(f"Data saved in: {output_csv_path}")

In [15]:
def label_spam_csv(csv):
    df = pd.read_csv(csv)
    df['Label'] = 1

    df.to_csv(csv, index=False, encoding='utf-8')
    print(f"Labeled CSV saved to: {csv}")

##### Exectute on VM

In [ ]:
spam_folder = r"C:\\Users\\ilian\\Downloads\\spam"
output_csv = paths.data.raw.data_mail.own.jannis_mail_csv

In [17]:
os.makedirs(spam_folder, exist_ok=True)
with open(os.path.join(spam_folder, 'test_sample.eml'), 'wb') as f:
    f.write(b"From: test@example.com\nTo: recipient@example.com\nSubject: Test Email with Umlauts\nDate: Fri, 16 Feb 2025 12:34:56 +0000\n\nHello, this is a test email with special characters like \xe4, \xfc, and \xf6.\nBest regards,\nTester")


In [18]:
process_spam_folder(spam_folder, output_csv)

Data saved in: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_detection\src\data_preperation\data\raw\jannis_mail.csv


In [27]:
label_spam_csv(output_csv)

Labeled CSV saved to: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_detection\data\raw\data_mail\own\jannis_mail.csv
